## Preprocessing

In [50]:
# Import our dependencies
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout, BatchNormalization
from tensorflow.keras.callbacks import ModelCheckpoint, EarlyStopping
import pandas as pd
import tensorflow as tf
from pathlib import Path
from google.colab import drive

#  Import and read the charity_data.csv.
import pandas as pd
application_df = pd.read_csv("https://static.bc-edx.com/data/dl-1-2/m21/lms/starter/charity_data.csv")
application_df.head()

,EIN,NAME,APPLICATION_TYPE,AFFILIATION,CLASSIFICATION,USE_CASE,ORGANIZATION,STATUS,INCOME_AMT,SPECIAL_CONSIDERATIONS,ASK_AMT,IS_SUCCESSFUL
0,10520599,BLUE KNIGHTS MOTORCYCLE CLUB,T10,Independent,C1000,ProductDev,Association,1,0,N,5000,1
1,10531628,AMERICAN CHESAPEAKE CLUB CHARITABLE TR,T3,Independent,C2000,Preservation,Co-operative,1,1-9999,N,108590,1
2,10547893,ST CLOUD PROFESSIONAL FIREFIGHTERS,T5,CompanySponsored,C3000,ProductDev,Association,1,0,N,5000,0
3,10553066,SOUTHSIDE ATHLETIC ASSOCIATION,T3,CompanySponsored,C2000,Preservation,Trust,1,10000-24999,N,6692,1
4,10556103,GENETIC RESEARCH INSTITUTE OF THE DESERT,T3,Independent,C1000,Heathcare,Trust,1,100000-499999,N,142590,1


In [51]:
# Separate numeric and non-numeric columns
numeric_columns = application_df.select_dtypes(include=['number']).columns.tolist()
non_numeric_columns = application_df.select_dtypes(exclude=['number']).columns.tolist()

In [52]:
# Check for columns with low variance
low_variance_columns = application_df[numeric_columns].var()[application_df[numeric_columns].var() < 0.1].index.tolist()
print(f"Numeric columns with low variance: {low_variance_columns}")

Numeric columns with low variance: ['STATUS']


In [53]:
# Check for high correlation among numeric features
correlation_matrix = application_df[numeric_columns].corr()
high_correlation_columns = correlation_matrix.columns[correlation_matrix.abs().max() > 0.9].tolist()
print(f"Numeric columns with high correlation: {high_correlation_columns}")

Numeric columns with high correlation: ['EIN', 'STATUS', 'ASK_AMT', 'IS_SUCCESSFUL']


In [54]:
# Drop the STATUS and ASK_AMT columns
application_df = application_df.drop(columns=['STATUS', 'ASK_AMT'])
application_df

,EIN,NAME,APPLICATION_TYPE,AFFILIATION,CLASSIFICATION,USE_CASE,ORGANIZATION,INCOME_AMT,SPECIAL_CONSIDERATIONS,IS_SUCCESSFUL
0,10520599,BLUE KNIGHTS MOTORCYCLE CLUB,T10,Independent,C1000,ProductDev,Association,0,N,1
1,10531628,AMERICAN CHESAPEAKE CLUB CHARITABLE TR,T3,Independent,C2000,Preservation,Co-operative,1-9999,N,1
2,10547893,ST CLOUD PROFESSIONAL FIREFIGHTERS,T5,CompanySponsored,C3000,ProductDev,Association,0,N,0
3,10553066,SOUTHSIDE ATHLETIC ASSOCIATION,T3,CompanySponsored,C2000,Preservation,Trust,10000-24999,N,1
4,10556103,GENETIC RESEARCH INSTITUTE OF THE DESERT,T3,Independent,C1000,Heathcare,Trust,100000-499999,N,1
...,...,...,...,...,...,...,...,...,...,...
34294,996009318,THE LIONS CLUB OF HONOLULU KAMEHAMEHA,T4,Independent,C1000,ProductDev,Association,0,N,0
34295,996010315,INTERNATIONAL ASSOCIATION OF LIONS CLUBS,T4,CompanySponsored,C3000,ProductDev,Association,0,N,0
34296,996012607,PTA HAWAII CONGRESS,T3,CompanySponsored,C2000,Preservation,Association,0,N,0
34297,996015768,AMERICAN FEDERATION OF GOVERNMENT EMPLOYEES LO...,T5,Independent,C3000,ProductDev,Association,0,N,1


In [55]:
# Drop the non-beneficial ID columns, 'EIN' and 'NAME'.
application_df = application_df.drop(columns=['EIN', 'NAME'])

In [56]:
# Determine the number of unique values in each column.
unique_values = application_df.nunique()
unique_values

APPLICATION_TYPE          17
AFFILIATION                6
CLASSIFICATION            71
USE_CASE                   5
ORGANIZATION               4
INCOME_AMT                 9
SPECIAL_CONSIDERATIONS     2
IS_SUCCESSFUL              2
dtype: int64

In [57]:
# Look at APPLICATION_TYPE value counts to identify and replace with "Other"
application_type_counts = application_df['APPLICATION_TYPE'].value_counts()
application_type_counts

APPLICATION_TYPE
T3     27037
T4      1542
T6      1216
T5      1173
T19     1065
T8       737
T7       725
T10      528
T9       156
T13       66
T12       27
T2        16
T25        3
T14        3
T29        2
T15        2
T17        1
Name: count, dtype: int64

In [58]:
# Function to bin rare occurrences in categorical columns
def bin_rare_occurrences(df, column_name, cutoff_value):
    value_counts = df[column_name].value_counts()
    to_replace = value_counts[value_counts < cutoff_value].index
    df[column_name] = df[column_name].replace(to_replace, "Other")
    return df

# Apply binning to 'APPLICATION_TYPE' and 'CLASSIFICATION'
application_df = bin_rare_occurrences(application_df, 'APPLICATION_TYPE', 500)
application_df = bin_rare_occurrences(application_df, 'CLASSIFICATION', 1000)

In [59]:
# Choose a cutoff value and create a list of application types to be replaced
# use the variable name `application_types_to_replace`

# Cutoff value
cutoff_value = 500

# Create a list of application types to be replaced
application_types_to_replace = application_type_counts[application_type_counts < cutoff_value].index.tolist()

# Replace in dataframe
for app in application_types_to_replace:
    application_df['APPLICATION_TYPE'] = application_df['APPLICATION_TYPE'].replace(app,"Other")

# Check to make sure replacement was successful
application_df['APPLICATION_TYPE'].value_counts()

APPLICATION_TYPE
T3       27037
T4        1542
T6        1216
T5        1173
T19       1065
T8         737
T7         725
T10        528
Other      276
Name: count, dtype: int64

In [60]:
# Look at CLASSIFICATION value counts to identify and replace with "Other"
classification_counts = application_df['CLASSIFICATION'].value_counts()
classification_counts.name = 'CLASSIFICATION'
print(classification_counts)

CLASSIFICATION
C1000    17326
C2000     6074
C1200     4837
Other     2261
C3000     1918
C2100     1883
Name: CLASSIFICATION, dtype: int64


In [61]:
# You may find it helpful to look at CLASSIFICATION value counts >1
classification_counts = application_df['CLASSIFICATION'].value_counts()

# Filter the value counts to only show those greater than 1
classification_counts_gt1 = classification_counts[classification_counts > 1]
classification_counts_gt1.name = 'CLASSIFICATION'
print(classification_counts_gt1)

CLASSIFICATION
C1000    17326
C2000     6074
C1200     4837
Other     2261
C3000     1918
C2100     1883
Name: CLASSIFICATION, dtype: int64


In [62]:
# Choose a cutoff value and create a list of classifications to be replaced
# use the variable name `classifications_to_replace`

# Cutoff Value
cutoff_value = 1000

# Create a list of classifications to be replaced
classifications_to_replace = classification_counts[classification_counts < cutoff_value].index.tolist()

# Replace in dataframe
for cls in classifications_to_replace:
    application_df['CLASSIFICATION'] = application_df['CLASSIFICATION'].replace(cls,"Other")

# Check to make sure replacement was successful
application_df['CLASSIFICATION'].value_counts()

CLASSIFICATION
C1000    17326
C2000     6074
C1200     4837
Other     2261
C3000     1918
C2100     1883
Name: count, dtype: int64

In [63]:
# Convert categorical data to numeric with `pd.get_dummies`
application_df_encoded = pd.get_dummies(application_df)
application_df_encoded

,IS_SUCCESSFUL,APPLICATION_TYPE_Other,APPLICATION_TYPE_T10,APPLICATION_TYPE_T19,APPLICATION_TYPE_T3,APPLICATION_TYPE_T4,APPLICATION_TYPE_T5,APPLICATION_TYPE_T6,APPLICATION_TYPE_T7,APPLICATION_TYPE_T8,...,INCOME_AMT_1-9999,INCOME_AMT_10000-24999,INCOME_AMT_100000-499999,INCOME_AMT_10M-50M,INCOME_AMT_1M-5M,INCOME_AMT_25000-99999,INCOME_AMT_50M+,INCOME_AMT_5M-10M,SPECIAL_CONSIDERATIONS_N,SPECIAL_CONSIDERATIONS_Y
0,1,False,True,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,True,False
1,1,False,False,False,True,False,False,False,False,False,...,True,False,False,False,False,False,False,False,True,False
2,0,False,False,False,False,False,True,False,False,False,...,False,False,False,False,False,False,False,False,True,False
3,1,False,False,False,True,False,False,False,False,False,...,False,True,False,False,False,False,False,False,True,False
4,1,False,False,False,True,False,False,False,False,False,...,False,False,True,False,False,False,False,False,True,False
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
34294,0,False,False,False,False,True,False,False,False,False,...,False,False,False,False,False,False,False,False,True,False
34295,0,False,False,False,False,True,False,False,False,False,...,False,False,False,False,False,False,False,False,True,False
34296,0,False,False,False,True,False,False,False,False,False,...,False,False,False,False,False,False,False,False,True,False
34297,1,False,False,False,False,False,True,False,False,False,...,False,False,False,False,False,False,False,False,True,False


In [64]:
# Split our preprocessed data into our features and target arrays
X = application_df_encoded.drop(columns=['IS_SUCCESSFUL'])
y = application_df_encoded['IS_SUCCESSFUL']

In [65]:
# Split the preprocessed data into a training and testing dataset
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

In [66]:
# Create a StandardScaler instances
scaler = StandardScaler()

# Fit the StandardScaler
X_scaler = scaler.fit(X_train)

# Scale the data
X_train_scaled = X_scaler.transform(X_train)
X_test_scaled = X_scaler.transform(X_test)

## Compile, Train and Evaluate the Model

In [77]:
# Define the model - deep neural net, i.e., the number of input features and hidden nodes for each layer.

# Define the model
model = Sequential()

# Input layer
model.add(Dense(units=512, activation='relu', input_dim=X_train_scaled.shape[1]))
model.add(BatchNormalization())
model.add(Dropout(0.5))

# First Hidden layer
model.add(Dense(units=256, activation='relu'))
model.add(BatchNormalization())
model.add(Dropout(0.4))

# Second Hidden layer
model.add(Dense(units=128, activation='relu'))
model.add(BatchNormalization())
model.add(Dropout(0.3))

# Third Hidden layer
model.add(Dense(units=64, activation='relu'))
model.add(BatchNormalization())
model.add(Dropout(0.3))

# Fourth Hidden layer
model.add(Dense(units=32, activation='relu'))
model.add(BatchNormalization())
model.add(Dropout(0.3))

# Output layer
model.add(Dense(units=1, activation='sigmoid'))

# Display the model summary
model.summary()

Model: "sequential_4"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense_23 (Dense)            (None, 512)               21504     
                                                                 
 batch_normalization_19 (Ba  (None, 512)               2048      
 tchNormalization)                                               
                                                                 
 dropout_19 (Dropout)        (None, 512)               0         
                                                                 
 dense_24 (Dense)            (None, 256)               131328    
                                                                 
 batch_normalization_20 (Ba  (None, 256)               1024      
 tchNormalization)                                               
                                                                 
 dropout_20 (Dropout)        (None, 256)              

In [78]:
# Compile the model
from tensorflow.keras.optimizers import Adam

adam = Adam(learning_rate=0.001)
model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

# Check the structure of the model
model.summary()

Model: "sequential_4"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense_23 (Dense)            (None, 512)               21504     
                                                                 
 batch_normalization_19 (Ba  (None, 512)               2048      
 tchNormalization)                                               
                                                                 
 dropout_19 (Dropout)        (None, 512)               0         
                                                                 
 dense_24 (Dense)            (None, 256)               131328    
                                                                 
 batch_normalization_20 (Ba  (None, 256)               1024      
 tchNormalization)                                               
                                                                 
 dropout_20 (Dropout)        (None, 256)              

In [79]:
# Implement Early Stopping to prevent overfitting
early_stopping = EarlyStopping(monitor='val_loss', patience=5, restore_best_weights=True)

In [80]:
# Train the model
X_train = X_train.astype('float32')
y_train = y_train.astype('int32')

history = model.fit(X_train_scaled, y_train, epochs=100, batch_size=128, validation_data=(X_test_scaled, y_test), verbose=1)

Epoch 1/100
215/215 [==============================] - 11s 30ms/step - loss: 0.6823 - accuracy: 0.6508 - val_loss: 0.5756 - val_accuracy: 0.7206
Epoch 2/100
215/215 [==============================] - 5s 25ms/step - loss: 0.6012 - accuracy: 0.7020 - val_loss: 0.5680 - val_accuracy: 0.7230
Epoch 3/100
215/215 [==============================] - 3s 15ms/step - loss: 0.5803 - accuracy: 0.7180 - val_loss: 0.5653 - val_accuracy: 0.7207
Epoch 4/100
215/215 [==============================] - 3s 15ms/step - loss: 0.5753 - accuracy: 0.7211 - val_loss: 0.5635 - val_accuracy: 0.7230
Epoch 5/100
215/215 [==============================] - 3s 15ms/step - loss: 0.5677 - accuracy: 0.7255 - val_loss: 0.5640 - val_accuracy: 0.7233
Epoch 6/100
215/215 [==============================] - 5s 24ms/step - loss: 0.5669 - accuracy: 0.7259 - val_loss: 0.5623 - val_accuracy: 0.7224
Epoch 7/100
215/215 [==============================] - 3s 15ms/step - loss: 0.5647 - accuracy: 0.7251 - val_loss: 0.5588 - val_accuracy

In [83]:
# Evaluate the model using the test data
model_loss, model_accuracy = model.evaluate(X_test_scaled,y_test,verbose=2)
print(f"Loss: {model_loss}, Accuracy: {model_accuracy}")

215/215 - 0s - loss: 0.5583 - accuracy: 0.7284 - 487ms/epoch - 2ms/step
Loss: 0.5582681894302368, Accuracy: 0.7284256815910339


In [84]:
# Export our model to HDF5 file
drive.mount('/content/gdrive')

model_path = '/content/gdrive/My Drive/AlphabetSoupCharityOptimization.h5'

model.save(model_path)

Drive already mounted at /content/gdrive; to attempt to forcibly remount, call drive.mount("/content/gdrive", force_remount=True).


/usr/local/lib/python3.10/dist-packages/keras/src/engine/training.py:3103: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(
